# DIOMIRA

## A nb to develop the cython version of DIOMIRA

In [1]:
from __future__ import print_function
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
%env IPYTHONDIR=/Users/jjgomezcadenas/Documents/Development/IPYTHON

env: IPYTHONDIR=/Users/jjgomezcadenas/Documents/Development/IPYTHON


In [3]:
%load_ext Cython

In [6]:
import pandas as pd
import numpy as np
import FEParam as FP
import tables

In [46]:
h5f =tables.open_file('/Users/jjgomezcadenas/Documents/Development/NEXT/data/Waveforms/25ns/WF_Na_ZLIB_test100_RWF.h5')

In [47]:
pmtrwf = h5f.root.RD.pmtrwf

### Python version

In [48]:

def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

def rebin_signal(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    
    rdata = []

    for j in range(pmtrd_.shape[1]):
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        twf = rebin_pmt_array(pmt, stride)
        
        rdata.append(twf)
    return np.array(rdata)

def rebin_signals(pmtrd, stride=40, list_of_events=[0]):
    """
    rebin the signals in a list of events
    """
    for event in list_of_events:
        rebin_signal(event,pmtrd, stride)
        
        

In [49]:
rebin_signals

<function __main__.rebin_signals>

In [50]:
%timeit rebin_signals(pmtrwf, stride=40, list_of_events=range(100))

1 loop, best of 3: 2.71 s per loop


### Cython version (just compile python)

In [52]:
%%cython
import pandas as pd
import numpy as np
import FEParam as FP
import tables

def rebin_pmt_array(a,stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    
    lenb = (len(a)+1)/int(stride)
    b = np.zeros(lenb)
    j=0
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

def rebin_signal(event_number,pmtrd_, stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    
    rdata = []

    for j in range(pmtrd_.shape[1]):
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        twf = rebin_pmt_array(pmt, stride)
        
        rdata.append(twf)
    return np.array(rdata)

def rebin_signals(pmtrd, stride=40, list_of_events=[0]):
    """
    rebin the signals in a list of events
    """
    for event in list_of_events:
        rebin_signal(event,pmtrd, stride)

In [53]:
rebin_signals

<function _cython_magic_3e2d21d9b55534167d1cdd6678f1621a.rebin_signals>

In [54]:
%timeit rebin_signals(pmtrwf, stride=40, list_of_events=range(100))

1 loop, best of 3: 2.7 s per loop


### Cython version (supposedly optimized)

In [58]:
%%cython
import pandas as pd
import numpy as np
import FEParam as FP
import tables

def rebin_pmt_array(float [:] a,int stride):
    """
    rebins pmt array a according to stride
    there is a rebin_array in util which uses
    lenb = (len(a))/int(stride)
    this version uses (lean(a)+1) to correct from the fact that the
    MCRD is 599999 channels (should be 600000)
    """
    cdef int lenb
    lenb = (len(a)+1)/stride
    b = np.zeros(lenb, dtype = np.float)
    cdef int j=0
    cdef int i
    for i in range(lenb):
        b[i] = np.sum(a[j:j+stride])
        j+= stride
    return b

def rebin_signal(int event_number, pmtrd_, int stride):
    """
    rebins the MCRD signal to produce TWF (pes, bins 25 ns)
    """
    
    #rdata = np.zeros(pmtrd_.shape[1], dtype = np.object)
    rdata = []
    cdef int j
    for j in range(pmtrd_.shape[1]):
        pmt = pmtrd_[event_number, j] #waveform for event event_number, PMT j
        twf = rebin_pmt_array(pmt, stride)
        
        rdata.append(twf)
    return np.array(rdata)

def rebin_signals(pmtrd, int stride, list_of_events):
    """
    rebin the signals in a list of events
    """
    cdef int event
    for event in list_of_events:
        rebin_signal(event,pmtrd, stride)

In [59]:
rebin_signals

<function _cython_magic_1a8e242b410a6e145bc050658b89d44f.rebin_signals>

In [60]:
%timeit rebin_signals(pmtrwf, stride=40, list_of_events=range(100))

1 loop, best of 3: 4.83 s per loop


### Cython does not improve over python code in this case!

In [ ]:
h5f.close()